In [1]:
# 라이브러리 로딩

from pandas_ods_reader import read_ods
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By

import argparse
import pandas as pd
import pprint
import sys
import time

In [2]:
import pandas as pd

In [3]:
def convert_string(x):
    if x == '신운영계':
        return 'tmslocalprd'
    elif x == '구운영계':
        return 'tmslocalover'

In [4]:
# 데이터 로딩
def read_fn():
# def read_fn(config):
    # 현재 위치에서 ods 파일 읽어 오기
    base_path = "./instance-owner-2021Oct10.ods"
    sheet = "VM_list"
    
#     base_path = config.ods_fn
#     sheet = config.ods_sheet # "Sheet1"

    df = read_ods(base_path, sheet, headers=True) #, columns=["cluster", "instance", "owner", "cellphone", "dept", "service"])
    df['cluster'] = df.DIV.apply(convert_string)
    df = df[['cluster', 'hostname', '담당자', '담당팀']]
    d = { 
        "cluster" : "cluster", 
        "hostname" : "instance", 
        "담당자" : "owner", 
        "담당팀" : "dept" }
    df.rename(columns= d, inplace = True)
    return df[:-6]

In [5]:
df = read_fn()

In [6]:
df

,cluster,instance,owner,dept
0,tmslocalprd,backup_restore/8a330160-a307-46e4-8858-db7ede8...,안진희,PaaS기술팀
1,tmslocalprd,bosh-deployments-exporter/d7e1334d-8042-4375-a...,안진희,PaaS기술팀
2,tmslocalprd,bosh-health-exporter/7bb3a9f7-5041-489c-a04f-a...,안진희,PaaS기술팀
3,tmslocalprd,ccsjennifer01,황성순,미들웨어기술팀
4,tmslocalprd,ccskraccountapigwp01,이승윤,고객데이터플랫폼서비스팀
...,...,...,...,...
1908,tmslocalover,ccskrvtwinmodelp02,김용진,차량데이터플랫폼서비스팀
1909,tmslocalover,ccskrvtwinprocessingp01,김용진,차량데이터플랫폼서비스팀
1910,tmslocalover,ccskrvtwinprocessingp02,김용진,차량데이터플랫폼서비스팀
1911,tmslocalover,ccskrwaflbedmzp01,이근호,클라우드운영센터


In [7]:
# 파일에서 읽은 정보 List 변수에 넣기
clusters = df['cluster'].tolist()
instances = df['instance'].tolist()
owners = df['owner'].tolist()
#     cellphones = df['cellphone'].tolist()
depts = df['dept'].tolist()
#     services = df['service'].tolist()

In [8]:
# 로그인
def login(driver, userid, passwd):
    # Ocloud admin 로그인 페이지로 이동
    driver.get('http://10.12.11.20:58080/admin')
    time.sleep(3)

    # UserID 입력
    username = driver.find_element_by_id("id_username")
    username.clear()
    username.send_keys(userid)

    # 암호 입력
    password = driver.find_element_by_id("id_password")
    password.clear()
    password.send_keys(passwd)

    # Login 버튼 클릭
    login = driver.find_element_by_xpath('//*[@id="login-form"]/div[3]/input').click()
    return login

In [9]:
# 웹브라우저 띄우기
driver = webdriver.Chrome()
time.sleep(3)

userid = 'COC'
passwd = '1234QWER'
        
ret = login(driver, userid, passwd)
print(ret)

None


In [10]:
count = 0
total = len(instances)

In [11]:
# 인스턴스 이름으로 찾기
def find_instance(driver, instance):
    searchbar = driver.find_element_by_id("searchbar")
    searchbar.clear()
    searchbar.send_keys(instance)
    driver.find_element_by_xpath('//*[@id="changelist-search"]/div/input[2]').click()
    time.sleep(3)

In [12]:
# Contact 정보 채우기
def fill_owner_info(driver, owner, description):
    instance_owner = driver.find_element_by_id("id_form-0-owner")
    instance_owner.clear()
    instance_owner.send_keys(owner)
    
    instance_description = driver.find_element_by_id("id_form-0-description")
    instance_description.clear()
    instance_description.send_keys(description)

# 저장하기
def save(driver):
    driver.find_element_by_xpath('//*[@id="changelist-form"]/p/input').click()

In [13]:
def gotocluster(driver, cluster, instance, owner, description):
    print ( "시작 : " + cluster + ',' + instance + ',' + description )

    if cluster == 'bigdata':
        # 클로스터로 이동
        driver.find_element_by_xpath('//*[@id="content-main"]/div[1]/table/tbody/tr[4]/td[2]/a').click()
        # 인스턴스 이름으로 찾기
        find_instance(driver, instance)
        # Contact 정보 채우기
        fill_owner_info(driver, owner, description)
        # 저장하기
        save(driver)
        time.sleep(1)
        
        # Ocloud admin 페이지로 이동
        driver.get('http://10.12.11.20:58080/admin')
        time.sleep(3)
        
    elif cluster =='bigdata2':
        # 클로스터로 이동
        driver.find_element_by_xpath('//*[@id="content-main"]/div[2]/table/tbody/tr[3]/td[2]/a').click()
                                      
        # 인스턴스 이름으로 찾기
        find_instance(driver, instance)
        # Contact 정보 채우기
        fill_owner_info(driver, owner, description)
        # 저장하기
        save(driver)
        time.sleep(1)
    
        # Ocloud admin 페이지로 이동
        driver.get('http://10.12.11.20:58080/admin')
        time.sleep(3)
        
    elif cluster == 'tmscnover':
        # 클로스터로 이동
        driver.find_element_by_xpath('//*[@id="content-main"]/div[9]/table/tbody/tr[4]/td[2]/a').click()
                                      
        # 인스턴스 이름으로 찾기
        find_instance(driver, instance)
        # Contact 정보 채우기
        fill_owner_info(driver, owner, description)
        # 저장하기
        save(driver)
        time.sleep(1)
    
        # Ocloud admin 페이지로 이동
        driver.get('http://10.12.11.20:58080/admin')
        time.sleep(3)

    elif cluster == 'tmslocalprd':
        # 클로스터로 이동
        driver.find_element_by_xpath('//*[@id="content-main"]/div[17]/table/tbody/tr[2]/td[2]/a').click()
                                      
        # 인스턴스 이름으로 찾기
        find_instance(driver, instance)
        # Contact 정보 채우기
        fill_owner_info(driver, owner, description)
        # 저장하기
        save(driver)
        time.sleep(1)
    
        # Ocloud admin 페이지로 이동
        driver.get('http://10.12.11.20:58080/admin')
        time.sleep(3)
        
    elif cluster == 'tmslocalover':
        # 클로스터로 이동
        driver.find_element_by_xpath('//*[@id="content-main"]/div[16]/table/tbody/tr[3]/td[2]/a').click()
                                      
        # 인스턴스 이름으로 찾기
        find_instance(driver, instance)
        # Contact 정보 채우기
        fill_owner_info(driver, owner, description)
        # 저장하기
        save(driver)
        time.sleep(1)
    
        # Ocloud admin 페이지로 이동
        driver.get('http://10.12.11.20:58080/admin')
        time.sleep(3)
                
    # 추가 클러스터에 적용하려면... 아래 조건문 사용하여 이동

#     elif cluster =='hkmc':
#         driver.find_element_by_xpath('//*[@id="content-main"]/div[3]/table/tbody/tr[2]/td[2]/a').click()
#     elif cluster =='hkmcdev':
#         driver.find_element_by_xpath('//*[@id="content-main"]/div[4]/table/tbody/tr[2]/td[2]/a').click()
#     elif cluster =='hmg':
#         driver.find_element_by_xpath('//*[@id="content-main"]/div[5]/table/tbody/tr[2]/td[2]/a').click()
#     elif cluster =='hmg2':
#         driver.find_element_by_xpath('//*[@id="content-main"]/div[6]/table/tbody/tr[3]/td[2]/a').click()
#     elif cluster =='rnd' :
#         driver.find_element_by_xpath('//*[@id="content-main"]/div[8]/table/tbody/tr[6]/td[2]/a').click()

    # 클러스터 이름이 맞지 않는 리스트
    else:
        print (cluster, instance)

In [14]:
df = pd.DataFrame(list(zip(clusters, instances, owners, depts)), columns = ['cluster', 'instance', 'owner', 'dept'] )

In [15]:
df2 = df.drop_duplicates()

In [22]:
df2.count()

cluster     1911
instance    1911
owner       1911
dept        1911
dtype: int64

In [29]:
df[df.instance == 'ccskrfmsdbp02']

,cluster,instance,owner,dept
465,tmslocalprd,ccskrfmsdbp02,박태성,CCS제어팀


In [25]:
df2.count()

cluster     1910
instance    1910
owner       1910
dept        1910
dtype: int64

In [28]:
df2.describe

<bound method NDFrame.describe of            cluster                                           instance owner  \
0      tmslocalprd  backup_restore/8a330160-a307-46e4-8858-db7ede8...   안진희   
1      tmslocalprd  bosh-deployments-exporter/d7e1334d-8042-4375-a...   안진희   
2      tmslocalprd  bosh-health-exporter/7bb3a9f7-5041-489c-a04f-a...   안진희   
3      tmslocalprd                                      ccsjennifer01   황성순   
4      tmslocalprd                               ccskraccountapigwp01   이승윤   
...            ...                                                ...   ...   
1908  tmslocalover                                 ccskrvtwinmodelp02   김용진   
1909  tmslocalover                            ccskrvtwinprocessingp01   김용진   
1910  tmslocalover                            ccskrvtwinprocessingp02   김용진   
1911  tmslocalover                                  ccskrwaflbedmzp01   이근호   
1912  tmslocalover                                  ccskrwaflbedmzp02   이근호   

              dep

In [30]:
# for cluster, instance, owner, cellphone, dept, service in zip(clusters, instances, owners, cellphones, depts, services):
for id, row in df2.iterrows():
    
    if id >= 465 :
        cluster  = row['cluster']
        instance = row['instance']
        owner    = row['owner']
        dept     = row['dept']

        description = ''
        if dept != None:
            description = str(owner) + ' / ' + str(dept)
        else:
            description = str(owner)

        try:
            gotocluster(driver, cluster, instance, owner, description)
            count += 1
        except:
            print ( "실패 : " + cluster + ',' + instance + ',' + description )
            driver.get('http://10.12.11.20:58080/admin')
            continue
        
print("총 갱신 레코드 / 갱신 성공(작업 수): ========== %d / %d" % (total, id+1) )

시작 : tmslocalprd,ccskrfmsdbp02,박태성 / CCS제어팀
실패 : tmslocalprd,ccskrfmsdbp02,박태성 / CCS제어팀
시작 : tmslocalprd,ccskrgdkchpdkdbp01,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpdkdbp02,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpeifsp01,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpeifsp02,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpelksp01,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpelksp02,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpelksp03,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpexifp01,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpexifp02,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpgccsp01,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpgccsp02,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpinifp01,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpinifp02,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpkmssp01,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpkmssp02,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpmsgqp01,김용공 / 국내모빌리티시스템개발팀
시작 : tmslocalprd,ccskrgdkchpmsgq

In [18]:
df2.tail

<bound method NDFrame.tail of            cluster                                           instance owner  \
0      tmslocalprd  backup_restore/8a330160-a307-46e4-8858-db7ede8...   안진희   
1      tmslocalprd  bosh-deployments-exporter/d7e1334d-8042-4375-a...   안진희   
2      tmslocalprd  bosh-health-exporter/7bb3a9f7-5041-489c-a04f-a...   안진희   
3      tmslocalprd                                      ccsjennifer01   황성순   
4      tmslocalprd                               ccskraccountapigwp01   이승윤   
...            ...                                                ...   ...   
1908  tmslocalover                                 ccskrvtwinmodelp02   김용진   
1909  tmslocalover                            ccskrvtwinprocessingp01   김용진   
1910  tmslocalover                            ccskrvtwinprocessingp02   김용진   
1911  tmslocalover                                  ccskrwaflbedmzp01   이근호   
1912  tmslocalover                                  ccskrwaflbedmzp02   이근호   

              dept  
